In [47]:
import time
start_time = time.time()

In [48]:
import glob
import pandas as pd
import scipy.io as sio
import numpy as np
import os

In [49]:
try:
    train = pd.read_pickle("train.pandas")
except (OSError, IOError) as e:
    print("Train features will be regenerated")
    train = pd.DataFrame({"filename": glob.glob("../input/train_*/*.mat")})
    train["type"] = train["filename"].str.extract('.*(\d+)\.mat$', expand=False)
    # fixing data
    fixed_files = pd.read_csv("../input/train_and_test_data_labels_safe.csv")
    train["image"] = train["filename"].apply(lambda x: x.split("/")[-1])
    train = pd.merge(train,fixed_files)
    train.drop(train.index[(train["safe"]==0)], inplace=True)
    train["type"] = train["class"]
    train.drop(["image", "safe", "class"], 1, inplace=True)
    train["type"] = train["type"].astype(object)
try:
    predict = pd.read_pickle("predict.pandas")
except (OSError, IOError) as e:
    print("Predict features will be regenerated")
    predict = pd.DataFrame({"filename": glob.glob("../input/test_*/*.mat")})

In [50]:
train.dtypes

avg0          object
avg1          object
avg10         object
avg11         object
avg12         object
avg13         object
avg14         object
avg15         object
avg2          object
avg3          object
avg4          object
avg5          object
avg6          object
avg7          object
avg8          object
avg9          object
dev0          object
dev1          object
dev10         object
dev11         object
dev12         object
dev13         object
dev14         object
dev15         object
dev2          object
dev3          object
dev4          object
dev5          object
dev6          object
dev7          object
               ...  
sum11         object
sum12         object
sum13         object
sum14         object
sum15         object
sum2          object
sum3          object
sum4          object
sum5          object
sum6          object
sum7          object
sum8          object
sum9          object
type          object
variance0     object
variance1     object
variance10   

In [51]:
train.head()

,avg0,avg1,avg10,avg11,avg12,avg13,avg14,avg15,avg2,avg3,...,variance14,variance15,variance2,variance3,variance4,variance5,variance6,variance7,variance8,variance9
0,-3.66211e-08,0.0303265,-0.0744333,0.0627337,0.108116,0.0733195,0.0428581,0.00116187,0.0449756,-0.00872452,...,221.616,179.889,483.037,1241.46,614.289,572.942,2787.3,183.646,229.928,822.817
11,1.05286e-06,-0.00321346,-0.272105,0.301897,0.388413,0.367831,0.250536,0.054799,0.105356,-0.0561697,...,5243.84,2164.75,3731.66,5747.93,3830.72,2971.17,2174.62,1713.1,3411.01,3200.49
12,-2.76693e-07,0.0285456,-0.196961,0.186818,0.486759,0.297833,0.257489,0.0510883,0.12269,-0.0803806,...,5585.83,2297.49,2138.69,3573.9,3614.81,1477.59,2118.85,1147.46,2224.61,3807.85
13,2.7771e-06,0.0821199,-0.224199,0.263479,0.30659,0.285726,0.223748,-0.0668703,0.21984,-0.0628751,...,1334.13,1036.31,1114.75,1576.42,860.43,1054.28,1239.95,2328.19,545.669,548.625
14,7.00887e-07,-0.0259684,-0.203042,0.243055,0.513649,0.35994,0.349966,0.0779311,0.0941403,-0.114259,...,8932.74,2772.74,2589.86,6347.44,1757.96,1379.18,2359.44,1480.53,2340.66,3890.05


In [52]:
predict.dtypes

avg0          object
avg1          object
avg10         object
avg11         object
avg12         object
avg13         object
avg14         object
avg15         object
avg2          object
avg3          object
avg4          object
avg5          object
avg6          object
avg7          object
avg8          object
avg9          object
dev0          object
dev1          object
dev10         object
dev11         object
dev12         object
dev13         object
dev14         object
dev15         object
dev2          object
dev3          object
dev4          object
dev5          object
dev6          object
dev7          object
               ...  
sum10         object
sum11         object
sum12         object
sum13         object
sum14         object
sum15         object
sum2          object
sum3          object
sum4          object
sum5          object
sum6          object
sum7          object
sum8          object
sum9          object
variance0     object
variance1     object
variance10   

In [53]:
def generate_features(data_frame_to_work):
    features = []
    #averages
    averages = ['avg{}'.format(i) for i in range(0, 16)]
    if not(set(data_frame_to_work.columns).issuperset(set(averages))):
        averages_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=averages)])
        print("Averages will be regenerated")
    else:
        averages_regenerate = False
        print("We have averages")
    features.append(averages_regenerate)
    #deviations
    devs = ['dev{}'.format(i) for i in range(0, 16)]
    if not(set(data_frame_to_work.columns).issuperset(set(devs))):
        deviations_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=devs)])
        print("Deviations will be regenerated")
    else:
        deviations_regenerate = False
        print("We have deviations")
    features.append(deviations_regenerate)
    #max
    maxs = ['max{}'.format(i) for i in range(0, 16)]
    if not(set(data_frame_to_work.columns).issuperset(set(maxs))):
        maximums_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=maxs)])
        print("Maximums will be regenerated")
    else:
        maximums_regenerate = False
        print("We have maximums")
    features.append(maximums_regenerate)
    #min
    mins = ['min{}'.format(i) for i in range(0, 16)]
    if not(set(data_frame_to_work.columns).issuperset(set(mins))):
        minimums_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=mins)])
        print("Minimums will be regenerated")
    else:
        minimums_regenerate = False
        print("We have minimums")
    features.append(minimums_regenerate)
    #spread
    spreads = ['spread{}'.format(i) for i in range(0, 16)]
    if not(set(data_frame_to_work.columns).issuperset(set(spreads))):
        spreads_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=spreads)])
        print("Spreads will be regenerated")
    else:
        spreads_regenerate = False
        print("We have spreads")
    features.append(spreads_regenerate)
    #median
    medians = ['median{}'.format(i) for i in range(0, 16)]
    if not(set(data_frame_to_work.columns).issuperset(set(medians))):
        medians_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=medians)])
        print("Medians will be regenerated")
    else:
        medians_regenerate = False
        print("We have medians")
    features.append(medians_regenerate)
    #sum
    sums = ['sum{}'.format(i) for i in range(0, 16)]
    if not(set(data_frame_to_work.columns).issuperset(set(sums))):
        sums_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=sums)])
        print("Sums will be regenerated")
    else:
        sums_regenerate = False
        print("We have sums")
    features.append(sums_regenerate)
    #variance
    variances = ['variance{}'.format(i) for i in range(0,16)]
    if not(set(data_frame_to_work.columns).issuperset(set(variances))):
        variances_regenerate = True
        data_frame_to_work = pd.concat([data_frame_to_work,pd.DataFrame(columns=variances)])
        print("Variances will be regenerated")
    else:
        variances_regenerate = False
        print("We have variances")
    features.append(variances_regenerate)
    print(features)
    if True in features:
        index_to_drop = []
        for row in data_frame_to_work.iterrows():
            filename = row[1]["filename"]
            try:
                tmp_data = sio.loadmat(filename)["dataStruct"][0][0][0]
            except ValueError:
                print("Something went wrong with loading file!")
                index_to_drop.append(row[0])
                continue
            if averages_regenerate == True:
                tmp_mean = np.mean(tmp_data, axis=0)
                for mean_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["avg"+str(mean_index)] = tmp_mean[mean_index]
            if deviations_regenerate == True:
                tmp_dev = np.std(tmp_data, axis=0)
                for std_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["dev"+str(std_index)] = tmp_dev[std_index]
            if maximums_regenerate == True:
                tmp_max = np.amax(tmp_data, axis=0)
                for max_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["max"+str(max_index)] = tmp_max[max_index]    
            if minimums_regenerate == True:
                tmp_min = np.amin(tmp_data, axis=0)
                for min_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["min"+str(min_index)] = tmp_min[min_index]
            if medians_regenerate == True:
                tmp_median = np.mean(tmp_data, axis=0)
                for median_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["median"+str(median_index)] = tmp_median[median_index]
            if sums_regenerate == True:
                tmp_sums = np.sum(tmp_data, axis=0)
                for sum_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["sum"+str(sum_index)] = tmp_sums[sum_index]
            if variances_regenerate == True:
                tmp_variances = np.var(tmp_data, axis=0)
                for variance_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["variance"+str(variance_index)] = tmp_variances[variance_index]
            if spreads_regenerate == True:
                for spread_index in range(0,16):
                    data_frame_to_work.loc[row[0]]["spread"+str(spread_index)] = data_frame_to_work.loc[row[0]]["max"+str(spread_index)] - data_frame_to_work.loc[row[0]]["min"+str(spread_index)]    
        data_frame_to_work.drop(data_frame_to_work.index[index_to_drop], inplace=True)
    return data_frame_to_work

In [54]:
train = generate_features(train)
train.to_pickle("train.pandas")

We have averages
We have deviations
We have maximums
We have minimums
We have spreads
We have medians
We have sums
We have variances
[False, False, False, False, False, False, False, False]


In [55]:
train.head()

,avg0,avg1,avg10,avg11,avg12,avg13,avg14,avg15,avg2,avg3,...,variance14,variance15,variance2,variance3,variance4,variance5,variance6,variance7,variance8,variance9
0,-3.66211e-08,0.0303265,-0.0744333,0.0627337,0.108116,0.0733195,0.0428581,0.00116187,0.0449756,-0.00872452,...,221.616,179.889,483.037,1241.46,614.289,572.942,2787.3,183.646,229.928,822.817
11,1.05286e-06,-0.00321346,-0.272105,0.301897,0.388413,0.367831,0.250536,0.054799,0.105356,-0.0561697,...,5243.84,2164.75,3731.66,5747.93,3830.72,2971.17,2174.62,1713.1,3411.01,3200.49
12,-2.76693e-07,0.0285456,-0.196961,0.186818,0.486759,0.297833,0.257489,0.0510883,0.12269,-0.0803806,...,5585.83,2297.49,2138.69,3573.9,3614.81,1477.59,2118.85,1147.46,2224.61,3807.85
13,2.7771e-06,0.0821199,-0.224199,0.263479,0.30659,0.285726,0.223748,-0.0668703,0.21984,-0.0628751,...,1334.13,1036.31,1114.75,1576.42,860.43,1054.28,1239.95,2328.19,545.669,548.625
14,7.00887e-07,-0.0259684,-0.203042,0.243055,0.513649,0.35994,0.349966,0.0779311,0.0941403,-0.114259,...,8932.74,2772.74,2589.86,6347.44,1757.96,1379.18,2359.44,1480.53,2340.66,3890.05


In [56]:
train.drop(train.index[ (train["avg0"]==0) | (train["avg1"]==0) | (train["avg2"]==0) | (train["avg3"]==0) | (train["avg4"]==0) | (train["avg5"]==0) | (train["avg6"]==0) | (train["avg7"]==0) | (train["avg8"]==0) | (train["avg9"]==0) | (train["avg10"]==0) | (train["avg11"]==0) | (train["avg12"]==0) | (train["avg13"]==0) | (train["avg14"]==0) | (train["avg15"]==0)], inplace=True)

In [57]:
train.drop(train[train.isnull().any(axis=1)].index, inplace=True)

In [58]:
predict = generate_features(predict)
predict.to_pickle("predict.pandas")
predict.head()

We have averages
We have deviations
We have maximums
We have minimums
We have spreads
We have medians
We have sums
We have variances
[False, False, False, False, False, False, False, False]


,avg0,avg1,avg10,avg11,avg12,avg13,avg14,avg15,avg2,avg3,...,variance14,variance15,variance2,variance3,variance4,variance5,variance6,variance7,variance8,variance9
0,2.90934e-07,0.0128468,-0.0308125,0.0261172,0.0416454,0.029716,0.0181867,0.00067063,0.0188994,-0.00512867,...,277.473,191.173,570.201,1088.38,671.257,432.555,233.285,1082.82,385.707,688.494
1,-6.51042e-08,0.000416679,-0.00756666,0.0107043,0.0114999,0.0124501,0.00794165,0.00293329,0.00376259,-0.00224598,...,131.701,66.0085,153.019,233.61,87.0432,41.9705,60.4017,66.639,90.637,306.978
2,-1.61947e-06,0.212879,-0.371078,0.546254,0.383356,0.479381,0.622148,-0.299318,0.489163,-0.127614,...,5555.39,3106.48,1710.03,3488.31,1736.13,3947.92,5489.59,10614.3,536.782,1500.95
3,3.38237e-07,-0.000166374,-0.0301423,0.0226587,0.11132,0.056109,0.0792797,0.0142955,0.0130836,-0.0365627,...,2192.41,537.254,535.141,3703.5,627.803,497.463,694.903,981.474,515.635,1637.19
4,-1.19832e-06,-0.0211958,-0.163725,0.216274,0.448783,0.3021,0.322762,0.0989273,0.0413812,-0.165313,...,6932.98,2350.74,2293.31,11065.9,2245.86,1835.09,1975.86,2620.42,1699.05,9451.49


In [59]:
from xgboost import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score

In [60]:
Y = train["type"].astype(int)
train.drop('type', axis=1, inplace=True)
# Y will be dataframe with classification only
# train will be dataframe with whole data without classification

In [61]:
Y.dtypes

dtype('int64')

In [62]:
train.drop('filename', axis=1, inplace=True)

In [63]:
train = train.apply(pd.to_numeric)

In [64]:
model = XGBClassifier()
model.fit(train, Y)
kfold = StratifiedKFold(Y, n_folds=10, random_state=7)
results = cross_val_score(model, train, Y, cv=kfold)

KeyboardInterrupt: 

In [65]:
y_pred = model.predict(train)

In [66]:
from sklearn.metrics import roc_auc_score
Y = Y.values.astype(int)
y_pred = y_pred.astype(int)
accuracy = roc_auc_score(Y, y_pred)
print("Accuracy: %.5f" % (accuracy))
if accuracy < 0.50921:
    print("Nothing special")
else:
    print("Might be worth submitting!")

Accuracy: 0.63587
Might be worth submitting!


### Let's try PCA

In [67]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
%matplotlib inline

In [68]:
train[train.isnull().any(axis=1)]

,avg0,avg1,avg10,avg11,avg12,avg13,avg14,avg15,avg2,avg3,...,variance14,variance15,variance2,variance3,variance4,variance5,variance6,variance7,variance8,variance9


In [69]:
X = train.values

In [70]:
X[3]

array([  2.77709955e-06,   8.21198598e-02,  -2.24198893e-01,
         2.63478518e-01,   3.06590289e-01,   2.85726398e-01,
         2.23747700e-01,  -6.68702796e-02,   2.19840422e-01,
        -6.28750920e-02,   7.72210583e-02,  -1.15089215e-01,
         1.62229091e-02,  -3.13068151e-01,  -7.81834796e-02,
        -2.15050071e-01,   3.43001404e+01,   3.13793488e+01,
         2.24680462e+01,   2.75001831e+01,   3.17401257e+01,
         3.66045532e+01,   3.65257378e+01,   3.21917648e+01,
         3.33879128e+01,   3.97040977e+01,   2.93330956e+01,
         3.24696808e+01,   3.52129822e+01,   4.82512703e+01,
         2.33595638e+01,   2.34227467e+01,   1.65432770e+02,
         1.57111740e+02,   1.90562927e+02,   4.49435028e+02,
         2.75874207e+02,   2.81948029e+02,   2.70754059e+02,
         1.78944702e+02,   1.79000000e+02,   2.17300812e+02,
         3.51027435e+02,   1.69707504e+02,   2.86836243e+02,
         1.96236343e+02,   1.69290176e+02,   1.80894730e+02,
         2.77709955e-06,

In [71]:
np.isfinite(X.all())

True

In [72]:
np.isinf(X.any())

False

In [73]:
np.isnan(np.sum(X))

False

In [74]:
X = scale(X)
pca = PCA(n_components=X.shape[1])
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=128, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [75]:
var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
#print var
#plt.plot(var)
#pick number of components which have just over 95% of variance
number_of_componenst = sum(i < 95 for i in var) + 1

In [76]:
potential_accuracy = 0.5
for components in xrange(1, X.shape[1]+1):
    pca = PCA(n_components=components)
    pca.fit(X)
    train_PCA = pca.transform(X)
    model_PCA = XGBClassifier()
    model_PCA.fit(train_PCA, Y)
    y_pred_PCA = model_PCA.predict(train_PCA)
    y_pred_PCA = y_pred_PCA.astype(int)
    accuracy_PCA = roc_auc_score(Y, y_pred_PCA)
    if accuracy_PCA > potential_accuracy:
        number_of_componenst = components
        potential_accuracy = accuracy_PCA
    print("For {} we have {}".format(components,accuracy_PCA))

For 1 we have 0.5
For 2 we have 0.516203703704
For 3 we have 0.539429266761
For 4 we have 0.53966478254
For 5 we have 0.547315774324
For 6 we have 0.568149107657
For 7 we have 0.568620139216
For 8 we have 0.570013062403
For 9 we have 0.570130820293
For 10 we have 0.573014253066
For 11 we have 0.585177116589
For 12 we have 0.584941600809
For 13 we have 0.587374173514
For 14 we have 0.587491931404
For 15 we have 0.585412632369
For 16 we have 0.586452281886
For 17 we have 0.585863492437
For 18 we have 0.581233862808
For 19 we have 0.58470608503


KeyboardInterrupt: 

In [77]:
number_of_componenst

14

In [78]:
pca = PCA(n_components=number_of_componenst)
pca.fit(X)
train_PCA = pca.transform(X)

In [79]:
model_PCA = XGBClassifier()
model_PCA.fit(train_PCA, Y)
#kfold_PCA = StratifiedKFold(Y, n_folds=10, random_state=7)
#results_PCA = cross_val_score(model_PCA, train_PCA, Y, cv=kfold_PCA)
y_pred_PCA = model_PCA.predict(train_PCA)

#Y = Y.values.astype(int)
y_pred_PCA = y_pred_PCA.astype(int)
accuracy_PCA = roc_auc_score(Y, y_pred_PCA)
print("Accuracy: %.5f" % (accuracy_PCA))
if accuracy_PCA < 0.50921:
    print("Nothing special_PCA")
else:
    print("Might be worth submitting_PCA!")

Accuracy: 0.58263
Might be worth submitting_PCA!


# Generate predictions for submission

In [80]:
submission = pd.DataFrame(predict['filename'])
predict.drop('filename', axis=1, inplace=True)
predict = predict.apply(pd.to_numeric)


In [81]:
predict.shape

(1908, 128)

In [82]:
if accuracy_PCA > accuracy:
    final_model = model_PCA
    predict = pca.transform(predict)
    print("Submitting after PCA")
else:
    final_model = model
    print("Submitting raw")

Submitting raw


In [83]:
submission['Class'] = final_model.predict(predict)

In [84]:
submission.rename(columns={'filename':'File'}, inplace=True)

In [85]:
submission['File'] = submission["File"].apply(lambda x: os.path.basename(x))

In [86]:
submission.to_csv("submission.csv", index=False)

In [87]:
submission["Class"].value_counts()

0    1900
1       8
Name: Class, dtype: int64

In [88]:
# Print python packages...
from pip.operations import freeze
x = freeze.freeze()
for p in x:
    print(p)
# ... and Python version
import sys
print(sys.version)

argparse==1.2.1
backports-abc==0.4
backports.shutil-get-terminal-size==1.0.0
backports.ssl-match-hostname==3.5.0.1
certifi==2016.9.26
configparser==3.5.0
cycler==0.10.0
decorator==4.0.10
entrypoints==0.2.2
enum34==1.1.6
functools32==3.2.3.post2
graphviz==0.5.1
ipykernel==4.5.0
ipython==5.1.0
ipython-genutils==0.1.0
ipywidgets==5.2.2
Jinja2==2.8
jsonschema==2.5.1
jupyter==1.0.0
jupyter-client==4.4.0
jupyter-console==5.0.0
jupyter-core==4.2.0
Keras==1.1.0
MarkupSafe==0.23
matplotlib==1.5.3
mistune==0.7.3
models==0.9.3
nbconvert==4.2.0
nbformat==4.1.0
notebook==4.2.3
numpy==1.11.1
pandas==0.18.0
pathlib2==2.1.0
patsy==0.4.1
pexpect==4.2.1
pickleshare==0.7.4
Pillow==3.4.2
pip==8.1.2
prompt-toolkit==1.0.7
ptyprocess==0.5.1
Pygments==2.1.3
pyparsing==2.1.9
Python==2.7.3
python-dateutil==2.5.3
pytz==2016.6.1
PyYAML==3.12
pyzmq==15.4.0
qtconsole==4.2.1
scikit-learn==0.18
scipy==0.17.0
setuptools==28.1.0
simplegeneric==0.8.1
singledispatch==3.4.0.3
six==1.10.0
sklearn==0.0
statsmodels==0.6.1
te

In [89]:
print((time.time() - start_time)/60)

0.940835750103


In [90]:
train

,avg0,avg1,avg10,avg11,avg12,avg13,avg14,avg15,avg2,avg3,...,variance14,variance15,variance2,variance3,variance4,variance5,variance6,variance7,variance8,variance9
0,-3.662110e-08,0.030327,-0.074433,0.062734,0.108116,0.073320,0.042858,0.001162,0.044976,-0.008725,...,221.616043,179.889496,483.037231,1241.460571,614.289490,572.942383,2787.301025,183.646378,229.928238,822.816650
11,1.052856e-06,-0.003213,-0.272105,0.301897,0.388413,0.367831,0.250536,0.054799,0.105356,-0.056170,...,5243.835938,2164.752441,3731.658691,5747.931152,3830.724609,2971.172363,2174.621826,1713.102417,3411.009277,3200.493164
12,-2.766927e-07,0.028546,-0.196961,0.186818,0.486759,0.297833,0.257489,0.051088,0.122690,-0.080381,...,5585.833984,2297.486572,2138.688721,3573.901123,3614.808838,1477.593872,2118.845947,1147.464233,2224.613770,3807.847168
13,2.777100e-06,0.082120,-0.224199,0.263479,0.306590,0.285726,0.223748,-0.066870,0.219840,-0.062875,...,1334.129639,1036.309692,1114.752808,1576.415527,860.430481,1054.280273,1239.954102,2328.185059,545.669250,548.625061
14,7.008870e-07,-0.025968,-0.203042,0.243055,0.513649,0.359940,0.349966,0.077931,0.094140,-0.114259,...,8932.743164,2772.737305,2589.860596,6347.443359,1757.957153,1379.181030,2359.442139,1480.532715,2340.661865,3890.052490
15,1.080322e-06,-0.013825,-0.194788,0.222752,0.439387,0.282866,0.281079,0.084277,0.093498,-0.119085,...,6557.306152,2071.582520,2051.152832,5098.180176,1303.211182,1110.936890,1784.448486,1187.813965,1504.313477,2758.895508
16,-1.044210e-06,-0.020982,-0.170198,0.187399,0.405759,0.268152,0.285705,0.069330,0.091072,-0.096792,...,6392.749023,2191.109131,1883.310913,4565.920898,1391.069336,1125.448975,1617.665527,1047.398315,1575.252686,2927.197266
17,-1.703898e-07,-0.000600,-0.024916,0.028963,0.051646,0.029129,0.038466,0.011221,0.008971,-0.014275,...,920.073181,295.392700,241.968353,689.261292,276.464874,168.697220,200.033340,172.532379,277.571930,380.675232
18,9.053548e-08,0.000571,-0.060505,0.084251,0.125654,0.077784,0.070404,0.020933,0.032463,-0.027351,...,1296.929810,533.993225,591.109314,1248.555786,520.080200,361.331909,401.990112,380.431335,569.354675,829.135010
23,-1.131185e-06,0.014733,-0.227161,0.186370,0.674568,0.366646,0.395320,0.063553,0.136853,-0.115857,...,9291.684570,3109.948242,2946.133545,4500.408691,7772.837402,2675.430176,3986.688965,1465.222168,2465.600830,5011.732422


In [91]:
train["sum0"]

0      -0.008789
11      0.252686
12     -0.066406
13      0.666504
14      0.168213
15      0.259277
16     -0.250610
17     -0.040894
18      0.021729
23     -0.271484
24     -0.210938
32      0.007446
33     -0.025391
34     -0.099609
35      0.316406
36      0.089844
37      0.081055
38     -0.220703
39     -0.288086
40      0.253906
41      0.266235
42      0.148438
43      0.302734
44      0.140564
45      0.064209
46     -0.253906
47      0.092773
48     -0.243164
49      0.175781
50     -0.010010
          ...   
5980    0.234131
5981    0.177734
5982    0.168945
5983   -0.032227
5984   -0.206055
5985   -0.053223
5986    0.220215
5987   -0.308594
5988   -0.674316
5989   -0.500000
5990   -0.256836
5991    0.353027
5992   -0.101807
5993   -0.029297
5994    0.316406
5995   -0.502441
5996    0.690918
5997    0.032715
5998    0.557129
5999   -0.206543
6003   -0.121582
6004    0.007812
6014   -0.178223
6015    0.144531
6026   -0.035645
6027    0.159180
6038   -0.191162
6039   -0.1308